In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# evaluate accuracy of the original networks before finetuning
from onnx2torch import convert
from stitchnet.stitchonnx.utils import accuracy_score_net,Net,get_score_net,load_cats_and_dogs_dset,load_dl,eval_original_model,get_macs_params_onnx,get_data_score
from stitchnet.stitchonnx.utils import accuracy_score_net_plants
from stitchnet.stitchonnx.utils import load_hf_train_val_dset
from stitchnet.stitchonnx.viz import draw_stitchNet_fromTuples
import torch
import os
from glob import glob
from tqdm import tqdm
from skl2onnx.helpers.onnx_helper import load_onnx_model
from pathlib import Path
import numpy as np
# data_score = get_data_score(32)
STITCH_BATCH_SIZE = 32

EVAL_BATCH_SIZE = 16
from stitchnet.stitchonnx.utils import load_hf_train_val_dset_with_test_split

dataset_train, dataset_val = load_hf_train_val_dset_with_test_split('beans', 
                                                                    train='validation', 
                                                                    val='test', 
                                                                    label='labels', 
                                                                    seed=47)

print(dataset_train['labels'][0:120])
# from transformers import AutoProcessor
from tqdm import tqdm
# p = AutoProcessor.from_pretrained('microsoft/resnet-50')
inps = []
# tgts = []
# for x in tqdm(dataset_train.take(STITCH_BATCH_SIZE)):
for x in tqdm(dataset_train.select(range(STITCH_BATCH_SIZE))):
    try:
        inps += x['pixel_values']
        # inps += p(x['image'])['pixel_values']
        # tgts += [x['label']]
    except:
        pass
data_score = np.stack(inps)

dataloaders = dict(
    train=load_dl(dataset_train, EVAL_BATCH_SIZE, shuffle=True),
    val=load_dl(dataset_val, EVAL_BATCH_SIZE, shuffle=False)
)
RESULT_NAME = f"_results/original_beans"
os.makedirs(RESULT_NAME, exist_ok=True)

modelnames = sorted(glob('_models/*.onnx'))
for i,modelname in tqdm(enumerate(modelnames)):
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    # if os.path.exists(f'_results/original/{namewithoutext}.txt'):
    #     continue
    model_onnx1 = load_onnx_model(modelname)
    macs, params = get_macs_params_onnx(model_onnx1)
    torch_model_1 = convert(model_onnx1)
    # valacc,trainacc = eval_original_model(torch_model_1, dataloaders)
    
    # get score
    fragmentFiles = sorted(glob(f'_models/fragments/net{i:03}/*.onnx'))
    onnxFragments = []
    js = []
    for j,fragmentFile in enumerate(fragmentFiles):
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
        js.append((i,j))
    net1 = Net(onnxFragments, i)
    score = get_score_net(net1, data_score)
    
    valacc = accuracy_score_net_plants(Net([model_onnx1]), dataset_val, dataset_train, bs=EVAL_BATCH_SIZE)
    trainacc = accuracy_score_net_plants(Net([model_onnx1]), dataset_train, dataset_train, bs=EVAL_BATCH_SIZE)
    print('VALACC:', valacc, 'TRAINACC:', trainacc)
    
    with open(f'{RESULT_NAME}/{namewithoutext}.txt', 'w') as f:
        f.write(f'{valacc},{trainacc},{macs},{params},{score},"{tuple(js)}"\n')
        
    draw_stitchNet_fromTuples(js, name=f'{RESULT_NAME}/{namewithoutext}')
        # f.write(f'{",".join([f"{x:.4}" for x in [valacc,trainacc,macs,params,score]])}\n')
    # break

Found cached dataset beans (/home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791)


  0%|          | 0/3 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-cac060a5360da545.arrow
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-fb581327019d6d2b.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-5f0b2533f6c38720.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-940888201d588dc5.arrow
Lo

tensor([1, 0, 0, 1, 1, 2, 0, 2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 2,
        0, 1, 1, 0, 0, 0, 2, 2, 1, 2, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, 1, 2, 2, 0,
        1, 1, 1, 0, 2, 0, 1, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 1, 2, 0, 2, 2, 1,
        2, 0, 1, 2, 0, 2, 2, 2, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 1, 1,
        2, 1, 2, 2, 2, 1, 2, 0, 1, 0, 0, 2, 2, 1, 0, 0, 0, 0, 2, 0, 2, 2, 0, 1])


100%|██████████| 32/32 [00:01<00:00, 20.10it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

...

(133, 1000) (133,)



  0%|          | 0/9 [00:00<?, ?it/s]2023-07-13 19:25:53.964816865 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {16,1000} for output 70
2023-07-13 19:25:53.994825644 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {16,1000} for output 70
2023-07-13 19:25:54.024171082 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {16,1000} for output 70

 33%|███▎      | 3/9 [00:00<00:00, 28.63it/s]2023-07-13 19:25:54.055578656 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {16,1000} for output 70
2023-07-13 19:25:54.084773992 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {16,1000} for output 70
2023-0

133
VALACC: 0.7890625 TRAINACC: 0.8421052631578947


/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_99 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_101 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_102 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.co

InferenceError: [TypeInferenceError] Inferred elem type differs from existing elem type: (INT64) vs (FLOAT)

In [ ]:
# from stitchnet.stitchonnx.utils import load_dl,load_onnx_model,load_cats_and_dogs_dset,execute_fragments,convert_imagenet_to_cat_dog_label
# import numpy as np
# from glob import glob
# from tqdm import tqdm

# dataset_val = load_cats_and_dogs_dset("test")
# fragmentFiles = sorted(glob(f'_models/fragments/net001/*.onnx'))
# onnxFragments = []
# js = []
# for j,fragmentFile in enumerate(fragmentFiles):
#     onnxFragment = load_onnx_model(fragmentFile)
#     onnxFragments.append(onnxFragment)

In [ ]:
print("DONE")